# **데이터 전처리**

1. **is_spike**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 + 2 × 표준편차보다 크면 1, 아니면 0, 위로 갑자기 튀는 것을 포착
2. **is_drop**: 전식당, 변할 수 있는 가능성을 학습하게끔 하는 것, 당일 수치가 최근 7일간 평균 − 2 × 표준편차보다 작으면 1, 아니면 0, 아래로 갑자기 튀는 것을 포착
3. **is_weekday_price**: 미라시아, 요금제가 주중 기준인지 여부 구분	메뉴명에 '주중'이 포함되면 1, 아니면 0, 주중 요금이 적용된 메뉴인지 여부
4. **is_weekend_price**: 미라시아, 요금제가 주말 기준인지 여부 구분	메뉴명에 '주말'이 포함되면 1, 아니면 0, 주말 요금이 적용된 메뉴인지 여부
5. **seasonal_index**: 전식당,

    • 1분기 (Q1): 1월 1일부터 3월 31일까지

    • 2분기 (Q2): 4월 1일부터 6월 30일까지

    • 3분기 (Q3): 7월 1일부터 9월 30일까지

    • 4분기 (Q4): 10월 1일부터 12월 31일까지

    월별 또는 분기별 매출 패턴 분석하여 생성, 분기별 매출 수치화
6. **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)** :
    
    6-1. **brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입에만 플래그를 세운다.

    6-2. **hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

## 데이터 불러오기 및 날짜 변환

In [1]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("train.csv")

# 날짜 변환
df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')


## **is_spike, is_drop(all)**

생성 목적: 변할 수 있는 가능성을 학습하게끔 하는 것

생성 방법:
1. is_spike:당일 수치가 최근 7일간 평균 + 2 × 표준편차보다 크면 1, 아니면 0
2. is_drop: 당일 수치가 최근 7일간 평균 − 2 × 표준편차보다 작으면 1, 아니면 0

In [2]:
def detect_spike_drop(group):
    group = group.sort_values('영업일자').copy()

    # 최근 7일 평균/표준편차 (당일 제외)
    rolling_mean = group['매출수량'].shift(1).rolling(window=7, min_periods=1).mean()
    rolling_std  = group['매출수량'].shift(1).rolling(window=7, min_periods=1).std(ddof=0)

    # 스파이크/드롭 플래그
    group['is_spike'] = (group['매출수량'] > (rolling_mean + 2 * rolling_std)).astype(int)
    group['is_drop']  = (group['매출수량'] < (rolling_mean - 2 * rolling_std)).astype(int)
    return group

# expand=True로 2개 컬럼(영업장명, 메뉴명)에 해당하는 임시 키 DataFrame 생성
keys = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

# keys[0] = 영업장명, keys[1] = 메뉴명  (df 컬럼으로 추가하지 않음!)
df = df.groupby([keys[0], keys[1]], group_keys=False).apply(detect_spike_drop)


## **is_weekday_price, is_weekend_price(미라시아)**

**is_weekday_price**:

생성 목적: 주중 요금이 적용된 메뉴인지 여부

생성 방법: 요금제가 주중 기준인지 여부 구분	메뉴명에 '주중'이 포함되면 1, 아니면 0

**is_weekend_price**:

생성 목적: 주말 요금이 적용된 메뉴인지 여부

생성 방법: 요금제가 주말 기준인지 여부 구분	메뉴명에 '주말'이 포함되면 1, 아니면 0

In [3]:
# 날짜 변환
if df['영업일자'].dtype == 'O':
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# '영업장명', '메뉴명' 없으면 분리 생성
if not {'영업장명','메뉴명'}.issubset(df.columns):
    df[['영업장명','메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

In [4]:
# 날짜 변환
if df['영업일자'].dtype == 'O':
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# '영업장명','메뉴명' 임시 분리 (df에 컬럼 추가 안 함)
split_tmp = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)
temp_store = split_tmp[0]           # 영업장명 (임시 시리즈)
temp_menu  = split_tmp[1]           # 메뉴명   (임시 시리즈)

# 플래그 컬럼 준비 (없으면 만들고, 있으면 그대로 씀)
if 'is_weekday_price' not in df.columns:
    df['is_weekday_price'] = 0
if 'is_weekend_price' not in df.columns:
    df['is_weekend_price'] = 0

# 미라시아 행만 대상
mask_mir = temp_store.eq('미라시아')
menu_clean = temp_menu.fillna('').str.replace(' ', '', regex=False)

# 주중/주말 포함 여부로 플래그 세팅 (해당 행에만 값 채움)
df.loc[mask_mir, 'is_weekday_price'] = menu_clean[mask_mir].str.contains('주중', na=False).astype(int)
df.loc[mask_mir, 'is_weekend_price'] = menu_clean[mask_mir].str.contains('주말', na=False).astype(int)

# (선택) 동시에 둘 다 1인 케이스 확인
both_mask = mask_mir & (df['is_weekday_price'].eq(1) & df['is_weekend_price'].eq(1))
# print(df.loc[both_mask, ['영업일자','영업장명_메뉴명','is_weekday_price','is_weekend_price']].head().to_string(index=False))

df.drop(columns=['영업장명','메뉴명'], errors='ignore', inplace=True)

# 확인 출력 (미라시아만)
# miracia_check = df[mask_mir][['영업장명_메뉴명','is_weekday_price','is_weekend_price']]
# print("=== 미라시아 요금제 구분 결과===")
# print(miracia_check.head(20).to_string(index=False))

# seasonal_index(all)

생성 목적: 분기별 매출 수치화

생성 방법:

• 1분기 (Q1): 1월 1일부터 3월 31일까지

• 2분기 (Q2): 4월 1일부터 6월 30일까지

• 3분기 (Q3): 7월 1일부터 9월 30일까지

• 4분기 (Q4): 10월 1일부터 12월 31일까지

분기별 매출 패턴 분석하여 누적합으로 생성

In [5]:
# 0) 날짜 보정
if df['영업일자'].dtype == 'O':
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# 1) '분기' → 'quarter'로 변경 (없으면 새로 생성)
#     Q1, Q2, Q3, Q4 형식으로 생성
df['quarter'] = df['영업일자'].dt.to_period('Q').astype(str).str[-2:]

In [6]:
import pandas as pd

# 0) 날짜 보정
if df['영업일자'].dtype == 'O':
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# 1) quarter 없으면 Q1~Q4 생성 (기존 있으면 건드리지 않음)
if 'quarter' not in df.columns:
    df['quarter'] = df['영업일자'].dt.to_period('Q').astype(str).str[-2:]

# 2) 임시로 매장/메뉴 분리(컬럼 추가 X)
keys = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)
g_store, g_menu = keys[0], keys[1]

# 3) 그룹별 전일까지 누적합 → 전체 열에 한 번에 대입 (정수 고정)
#    - 정렬은 누적 순서만 위해 잠깐 사용, 결과는 원래 인덱스로 돌아옴
df_sorted = df.sort_values(['quarter', '영업일자']).copy()
seasonal_series = (
    df_sorted
      .groupby([g_store.reindex(df_sorted.index),
                g_menu.reindex(df_sorted.index),
                df_sorted['quarter']], sort=False)['매출수량']
      .transform(lambda s: s.shift(1).cumsum())
)

df['seasonal_index'] = (
    seasonal_series.reindex(df_sorted.index)
                  .reindex(df.index)
                  .fillna(0)
                  .astype('int64')
)

# **미라시아 단체 관련 변수( brunch_flag, hallroom_flag)**

**brunch_flag**: 단체 브런치 메뉴 매출이 생긴 날의 플래그, 연회장_룸타입('Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS 2')에만 플래그를 세운다.

**hallroom_flag**: 연회장_룸타입 매출이 생긴 날의 플래그, (단체)브런치주중 36,000 에만 플래그를 세운다.

In [7]:
TARGET = '미라시아_(단체)브런치주중 36,000'
HALL_ROOMS = {'Grand Ballroom', 'Convention Hall', 'Conference L', 'Conference M', 'OPUS'}

# 날짜형 변환
if df['영업일자'].dtype == 'O':
    df['영업일자'] = pd.to_datetime(df['영업일자'], errors='coerce')

# '영업장명_메뉴명' 분리
tokens = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)
store0 = tokens[0].astype(str).str.strip()   # 예: '연회장', '미라시아', ...
store1 = tokens[1].astype(str).str.strip()   # 예: 'Grand Ballroom', '(단체)브런치주중 36,000', ...

# 1) 연회장 매출 발생 날짜
hall_mask = (store0.eq('연회장')) & (store1.isin(HALL_ROOMS)) & (df['매출수량'] > 0)
hall_dates = df.loc[hall_mask, '영업일자'].unique()

# 2) 브런치 매출 발생 날짜
brunch_mask = df['영업장명_메뉴명'].eq(TARGET) & (df['매출수량'] > 0)
brunch_dates = df.loc[brunch_mask, '영업일자'].unique()

# 3) 플래그 생성 (반대로 반영)
# 초기화: 전부 0
df['brunch_flag'] = 0      # ← 연회장 라인에 찍힘 (브런치 매출 발생일 기준)
df['hallroom_flag'] = 0    # ← 미라시아 단체 브런치 라인에 찍힘 (연회장 매출 발생일 기준)

# A) 단체 브런치 매출 발생일 → "연회장_*" 행에 brunch_flag=1
df.loc[hall_mask & df['영업일자'].isin(brunch_dates), 'brunch_flag'] = 1

# B) 연회장 매출 발생일 → "미라시아_(단체)브런치주중 36,000" 행에 hallroom_flag=1
target_row_mask = df['영업장명_메뉴명'].eq(TARGET)
df.loc[target_row_mask & df['영업일자'].isin(hall_dates), 'hallroom_flag'] = 1

# 확인
# print(df.loc[df['영업장명_메뉴명'].eq(TARGET),
#              ['영업일자', '영업장명_메뉴명', '매출수량', 'hallroom_flag']].head())

# print(df.loc[hall_mask,
#              ['영업일자', '영업장명_메뉴명', '매출수량', 'brunch_flag']].head())

In [8]:
df

,영업일자,영업장명_메뉴명,매출수량,is_spike,is_drop,is_weekday_price,is_weekend_price,quarter,seasonal_index,brunch_flag,hallroom_flag
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,0,0,0,0,Q1,0,0,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,0,0,0,0,Q1,0,0,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,0,0,0,0,Q1,0,0,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,0,0,0,0,Q1,0,0,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,0,0,0,0,Q1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,0,0,0,0,Q2,5709,0,0
102672,2024-06-12,화담숲카페_현미뻥스크림,10,0,0,0,0,Q2,5721,0,0
102673,2024-06-13,화담숲카페_현미뻥스크림,14,0,0,0,0,Q2,5731,0,0
102674,2024-06-14,화담숲카페_현미뻥스크림,12,0,0,0,0,Q2,5745,0,0


In [9]:
# df를 CSV로 저장
df.to_csv("output.csv", index=False, encoding="utf-8-sig")